In [1]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from pyspark.sql.functions import col, when

In [2]:
spark = (
    SparkSession.builder
    .appName("Local-ETL-Test")
    .master("local[*]")
    .config("spark.driver.memory", "2g")
    .config("spark.sql.files.maxPartitionBytes", 256 * 1024 * 1024) # 256 * 1024 * 1024 bytes
    .config("spark.sql.shuffle.partitions", "200") # 200 partitions for shuffle operations
    .getOrCreate()
)
spark.sparkContext.setLogLevel("ERROR")

In [4]:
df = spark.read.json("/data/destination/log_search/category/mapping_std/*.json")
logsearchm6= spark.read.option("header", "True").csv("/data/destination/log_search/month=6/*.csv")
logsearchm7= spark.read.option("header", "True").csv("/data/destination/log_search/month=7/*.csv")

In [5]:
df.show(truncate= False)

+------------+-----------------------------------------------------+
|category_std|keyword                                              |
+------------+-----------------------------------------------------+
|Other       |Cửa hàng tiện                                        |
|Other       |phố mùa 3                                            |
|Other       |Người phán sử tập 1                                  |
|Other       |seal team                                            |
|Other       |Đá banh vn                                           |
|Other       |karaoke em vẫn lầm tin anh                           |
|Other       | ngụy tạo                                            |
|Other       |community life                                       |
|Other       |hoàng cung phim 2009                                 |
|Other       |CONAN VA VIEN DAN DO                                 |
|Other       |I AM                                                 |
|Other       |mở ca cải lương     

In [6]:
def join_df(data1, data2, key1, key2, select_cols):
    df = data1.join(
        data2,
        data1[key1] == data2[key2],
        how="left"
    )
    return df.select(*select_cols)


In [7]:
m6 = join_df(
    logsearchm6,
    df,
    "mostWatch",
    "keyword",
    ["user_id", "mostWatch", "category_std"]
)

m7 = join_df(
    logsearchm7,
    df,
    "mostWatch",
    "keyword",
    ["user_id", "mostWatch", "category_std"]
)


In [8]:
result = (
    m6.alias("a")
    .join(
        m7.alias("b"),
        col("a.user_id") == col("b.user_id"),
        how="left"
    )
    .select(
        col("a.user_id").alias("user_id"),
        col("a.mostWatch").alias("mostWatch_m6"),
        col("b.mostWatch").alias("mostWatch_m7"),
        col("a.category_std").alias("category_m6"),
        col("b.category_std").alias("category_m7")
    )
)

In [9]:
result.show(truncate= False)

+--------+-------------------------+-------------------+-----------+-----------+
|user_id |mostWatch_m6             |mostWatch_m7       |category_m6|category_m7|
+--------+-------------------------+-------------------+-----------+-----------+
|06441094|Má tôi                   |NULL               |Other      |NULL       |
|0839625 |tuyết trung hãn          |giác               |Other      |Other      |
|1599286 |trường an như cố         |master in the house|Other      |Other      |
|1635951 |cảmh sát                 |NULL               |Horror     |NULL       |
|2256205 |bóng đá Việt Nam và arap |NULL               |Sports     |NULL       |
|229788  |titanic                  |NULL               |Other      |NULL       |
|238119  |tennis                   |NULL               |Other      |NULL       |
|41817074|gia dinh nha loud        |NULL               |Other      |NULL       |
|41904596|TO TINH HOANG MY   TAP 81|NULL               |Other      |NULL       |
|43154813|ANH HUNG KHIEN    

In [10]:
result = result.withColumn(
    "taste",
    when(col("category_m6") == col("category_m7"), F.lit("Unchanged"))
    .otherwise(F.lit("Changed"))
)

In [11]:
result.show(truncate= False)

+--------+-------------------------+-------------------+-----------+-----------+---------+
|user_id |mostWatch_m6             |mostWatch_m7       |category_m6|category_m7|taste    |
+--------+-------------------------+-------------------+-----------+-----------+---------+
|06441094|Má tôi                   |NULL               |Other      |NULL       |Changed  |
|0839625 |tuyết trung hãn          |giác               |Other      |Other      |Unchanged|
|1599286 |trường an như cố         |master in the house|Other      |Other      |Unchanged|
|1635951 |cảmh sát                 |NULL               |Horror     |NULL       |Changed  |
|2256205 |bóng đá Việt Nam và arap |NULL               |Sports     |NULL       |Changed  |
|229788  |titanic                  |NULL               |Other      |NULL       |Changed  |
|238119  |tennis                   |NULL               |Other      |NULL       |Changed  |
|41817074|gia dinh nha loud        |NULL               |Other      |NULL       |Changed  |